In [1]:
%%writefile exemplo1.cu
#include <thrust/device_vector.h> // Inclui a biblioteca para vetores na GPU (device)
#include <thrust/host_vector.h>   // Inclui a biblioteca para vetores na CPU (host)
#include <iostream>               // Biblioteca padrão para entrada e saída de dados

int main() {
    // Cria um vetor na CPU (host) com 5 elementos, todos inicializados com 0
    thrust::host_vector<double> host(5, 0);
    host[4] = 35; // Altera o último elemento do vetor para 35

    /* Os dados do vetor 'host' são copiados para a GPU,
       criando um vetor equivalente na GPU (device) */
    thrust::device_vector<double> dev(host);

    /* Altera o vetor na CPU, mas não afeta o vetor na GPU,
       pois a cópia já foi feita anteriormente */
    host[2] = 12; // Altera o terceiro elemento do vetor na CPU para 12

    // Exibe os elementos do vetor na CPU
    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // Acesso rápido aos elementos na CPU
    }
    printf("\n");

    // Exibe os elementos do vetor na GPU
    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // Acesso aos elementos na GPU é lento, pois os dados estão na GPU
    }
    printf("\n");
}

Writing exemplo1.cu


In [3]:
!nvcc -arch=sm_75 -std=c++14 exemplo1.cu -o exemplo1

In [4]:
!./exemplo1

Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


In [5]:
%%writefile exemplo2.cu
#include <thrust/device_vector.h>              // Inclui a biblioteca para vetores na GPU
#include <thrust/host_vector.h>                // Inclui a biblioteca para vetores na CPU
#include <thrust/sequence.h>                   // Inclui a função para preencher vetores com uma sequência de números
#include <thrust/functional.h>                 // Inclui operações matemáticas padrão, como soma e multiplicação
#include <thrust/transform.h>                  // Inclui a função para transformar vetores com operações ponto a ponto
#include <thrust/iterator/constant_iterator.h> // Inclui o iterador constante necessário
#include <iostream>

int main() {
    // Cria um vetor na GPU (device_vector) com 10 elementos, todos inicializados com 0
    thrust::device_vector<double> V1(10, 0);
    // Preenche V1 com uma sequência de números: {0, 1, 2, ..., 9}
    thrust::sequence(V1.begin(), V1.end());

    // Cria um vetor na GPU com 5 elementos, todos inicializados com 0
    thrust::device_vector<double> V2(5, 0);
    // Preenche os dois primeiros elementos de V2 com 5.5: {5.5, 5.5, 0, 0, 0}
    thrust::fill(V2.begin(), V2.begin() + 2, 5.5);
    // Preenche os elementos restantes de V2 com 10: {5.5, 5.5, 10, 10, 10}
    thrust::fill(V2.begin() + 2, V2.end(), 10);

    // Cria dois vetores na GPU com 10 elementos, para armazenar resultados das operações
    thrust::device_vector<double> V3(10); // Vetor para armazenar o resultado da soma
    thrust::device_vector<double> V4(10); // Vetor para armazenar o resultado da multiplicação

    // Aplica a operação de soma elemento por elemento entre V1 e V2 e armazena o resultado em V3
    // Como V2 tem menos elementos, o restante de V1 é somado com zeros (elemento padrão).
    // Resultado: V3 = {0+5.5, 1+5.5, 2+10, 3+10, 4+10, 5+0, ..., 9+0}
    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());

    // Aplica multiplicação de V1 com o valor constante 0.5
    // e armazenaria o resultado em V4.
    thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    // Imprime os elementos de V1
    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V1
    }
    printf("\n");

    // Imprime os elementos de V2
    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V2
    }
    printf("\n");

    // Imprime os elementos de V3
    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V3
    }
    printf("\n");

    // Imprime o vetor V4, que foi criado mas não utilizado.
     printf("V4: ");
    for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
         std::cout << *i << " ";
    }
    printf("\n");

    return 0;
}

Writing exemplo2.cu


In [6]:
!nvcc -arch=sm_75 -std=c++14 exemplo2.cu -o exemplo2

In [7]:
!./exemplo2


V1: 0 1 2 3 4 5 6 7 8 9 
V2: 5.5 5.5 10 10 10 
V3: 5.5 6.5 12 13 14 5 6 7 8 9 
V4: 0 0.5 1 1.5 2 2.5 3 3.5 4 4.5 


In [8]:
%%writefile stocks.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    return 0;
}

Writing stocks.cu


In [9]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

In [10]:
!./stocks < stocks-google.txt

/bin/bash: line 1: stocks-google.txt: No such file or directory


In [14]:
%%writefile reducao.cu
#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <iostream>

int main(){
    // Criando um vetor na GPU com 5 elementos: {10, 20, 30, 40, 50}
    thrust::device_vector<int> vec_gpu(5);
    thrust::sequence(vec_gpu.begin(), vec_gpu.end(), 10, 10);

    // Calculando a soma dos elementos
    int soma = thrust::reduce(vec_gpu.begin(), vec_gpu.end(), 0, thrust::plus<int>());

    std::cout << "Soma dos elementos: " << soma << std::endl; // Saída: 150

    return 0;
}

Writing reducao.cu


In [16]:
%%writefile exe01.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <iostream>
#include <chrono>
#include <fstream>

int main() {
    int n = 3112;
    thrust::host_vector<double> host(n); // Vetor na CPU para armazenar os dados

    // Abrir o arquivo
    std::ifstream file("stocks-google.txt");
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo" << std::endl;
        return -1;
    }

    // Ler os valores do arquivo e armazenar no host_vector
    for (int i = 0; i < n; i++) {
        file >> host[i];
    }
    file.close();

    // Medir o tempo de cópia dos dados para a GPU
    auto start = std::chrono::steady_clock::now();
    thrust::device_vector<double> dev = host; // Copiando os dados para a GPU
    auto end = std::chrono::steady_clock::now();

    std::chrono::duration<double> diff = end - start;
    std::cout << "Tempo de CÓPIA (em segundos): " << diff.count() << std::endl;

    return 0;
}


Writing exe01.cu


In [17]:
!nvcc -arch=sm_75 -std=c++14 exe01.cu -o exe01

In [18]:
!./exe01

Tempo de CÓPIA (em segundos): 0.20149


In [38]:
%%writefile exe02.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <iostream>
#include <fstream>
#include <limits> // Para o uso de infinity

int main() {
    // Suponha que temos um arquivo de dados e vamos ler os dados no host_vector
    int n = 3112; 
    int last_year_days = 365; // Últimos 365 dias

    // Vetor na CPU (host_vector) para armazenar os dados
    thrust::host_vector<double> host(n);

    // Abrir o arquivo e ler os valores
    std::ifstream file("stocks-google.txt");
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo" << std::endl;
        return -1;
    }

    for (int i = 0; i < n; i++) {
        file >> host[i]; // Ler o valor do arquivo e armazenar no host_vector
    }
    file.close();

    // Transferir os dados do host_vector para o device_vector
    thrust::device_vector<double> dev = host; // Transferindo para a GPU

    // Cálculo do preço médio dos últimos 10 anos (período total)
    double soma_total = thrust::reduce(dev.begin(), dev.end(), 0.0, thrust::plus<double>());
    double media_total = soma_total / n;
    std::cout << "Preço médio dos últimos 10 anos: " << media_total << std::endl;

    // Cálculo do preço médio dos últimos 365 dias
    double soma_365 = thrust::reduce(dev.end() - last_year_days, dev.end(), 0.0, thrust::plus<double>());
    double media_365 = soma_365 / last_year_days;
    std::cout << "Preço médio dos últimos 365 dias: " << media_365 << std::endl;

    // Encontrar o maior e o menor preço no período total
    double max_total = thrust::reduce(dev.begin(), dev.end(), -std::numeric_limits<double>::infinity(), thrust::maximum<double>());
    double min_total = thrust::reduce(dev.begin(), dev.end(), std::numeric_limits<double>::infinity(), thrust::minimum<double>());
    std::cout << "Maior preço dos últimos 10 anos: " << max_total << std::endl;
    std::cout << "Menor preço dos últimos 10 anos: " << min_total << std::endl;

    // Encontrar o maior e o menor preço nos últimos 365 dias
    double max_365 = thrust::reduce(dev.end() - last_year_days, dev.end(), -std::numeric_limits<double>::infinity(), thrust::maximum<double>());
    double min_365 = thrust::reduce(dev.end() - last_year_days, dev.end(), std::numeric_limits<double>::infinity(), thrust::minimum<double>());
    std::cout << "Maior preço dos últimos 365 dias: " << max_365 << std::endl;
    std::cout << "Menor preço dos últimos 365 dias: " << min_365 << std::endl;

    return 0;
}


Overwriting exe02.cu


In [39]:
!nvcc -arch=sm_75 -std=c++14 exe02.cu -o exe02

In [40]:
!./exe02

Preço médio dos últimos 10 anos: 1580.08
Preço médio dos últimos 365 dias: 1615.81
Maior preço dos últimos 10 anos: 2200
Menor preço dos últimos 10 anos: 1015.24
Maior preço dos últimos 365 dias: 2200
Menor preço dos últimos 365 dias: 1060.5


In [41]:
%%writefile exe03.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <iostream>
#include <fstream>

int main() {
    int n = 3520;

    // Vetores na CPU (host_vector) para armazenar os preços da Apple e da Microsoft
    thrust::host_vector<double> host_AAPL(n);
    thrust::host_vector<double> host_MSFT(n);

    // Abrir o arquivo e ler os valores
    std::ifstream file("stocks2.txt");
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo" << std::endl;
        return -1;
    }

    // Ler os valores do arquivo, onde cada linha contém dois preços separados por vírgula
    for (int i = 0; i < n; i++) {
        char comma;
        file >> host_AAPL[i] >> comma >> host_MSFT[i]; // Lê o valor da Apple, depois da Microsoft
    }
    file.close();

    // Transferir os dados dos host_vectors para os device_vectors
    thrust::device_vector<double> dev_AAPL = host_AAPL;
    thrust::device_vector<double> dev_MSFT = host_MSFT;

    // Vetor para armazenar as diferenças
    thrust::device_vector<double> differences(n);

    // Calcular a diferença ponto a ponto entre os preços da Apple e Microsoft
    thrust::transform(dev_AAPL.begin(), dev_AAPL.end(), dev_MSFT.begin(), differences.begin(), thrust::minus<double>());

    // Calcular a soma das diferenças
    double soma_dif = thrust::reduce(differences.begin(), differences.end(), 0.0, thrust::plus<double>());

    // Calcular a média das diferenças
    double media_dif = soma_dif / n;

    std::cout << "Média das diferenças entre os preços das ações da Apple e Microsoft: " << media_dif << std::endl;

    return 0;
}


Overwriting exe03.cu


In [42]:
!nvcc -arch=sm_75 -std=c++14 exe03.cu -o exe03

In [43]:
!./exe03

Média das diferenças entre os preços das ações da Apple e Microsoft: -265.232
